# Parsing a CSGO demofile¶
##### Last Updated: October 7, 2021

The csgo package allows for the parsing of CSGO demofiles into JSON or tabular format, which makes for easy analysis. To install the package, clone the Github repository, navigate to the root directory and run `python setup.py install`. Make sure you have installed at least [Golang](https://golang.org/) version 1.14. Please see expanded documentation [here](https://github.com/pnxenopoulos/csgo/tree/main/csgo/docs).

##### What is a demofile?
Each CSGO match typically generates recordings of the game called a demofile. Every map in a match will generate its own demofile. This demofile contains a serialization of the data transferred between the host (game server) and its clients (players). Data transferred to and from the server occurs at a predefined tick rate, which defines when client inputs are resolved with the server. For professional games, the server tick rate is usually 128 ticks per second, meaning each tick represents around 7.8 milliseconds. Client inputs represent player actions, such as movement, attacks or grenade throws. Non-client events also register in the demofile, such as round starts and ends.

Since CSGO demofiles are essentially data transferred between the clients and the game server, the data is simply stored as a text of sequential set of events with no contextual information, such as round or map location. Thus, due to the highly unstructured nature of these low level CSGO data streams, akin to log files, performing any complex analytic tasks becomes impossible without modeling this data formally into a useful data structure. The csgo package addresses this need.

##### How do I get demofiles?¶
One of the most common ways to obtain demofiles is through the site [HLTV](https://hltv.org). If you navigate to a match page, an hour or so after the match has concluded, it will often contain a link to the demofiles for that match. The link is a `.rar` compressed directory containing a demo for each map in the match.

# Initializing the parser
Let's consider the demo from a match between OG and Natus Vincere (NaVi), which we can [download from HLTV](https://www.hltv.org/matches/2344822/og-vs-natus-vincere-blast-premier-fall-series-2020). If we download the compressed demofile directory (a few hundred MB), there are three files:

- `og-vs-natus-vincere-m1-dust2.dem`
- `og-vs-natus-vincere-m2-mirage.dem`
- `og-vs-natus-vincere-m3-inferno.dem`

In order to parse one of the demofiles, you have to instantiate a DemoParser object and pass it the path to the demofile. The parser can automatically infer a `demo_id` from the filename, but you can also specify one using the argument. This name is use to title the JSON output file from the parser. We will use a `demo_id` of `OG-NaVi-BLAST2020`. 

There are a few numeric arguments to specify as one. The primary one is the `parse_rate`, or, how often do we want to parse a frame? This does _not_ affect how often we parse events, like kills or damages, only player positioning. Setting a parse rate of 1 means we are acquiring every frame (this is _very_ slow). Usually, 128 or 64 will work, depending on the tick rate of the demo. You can also specify the trade time (how many seconds inbetween trades) using the `trade_time` argument. Usually, a few seconds is good -- the default is 5. Finally, you can specify the round buy style (either "csgo" or "hltv", default is "hltv"). This argument changes the string for the round buy type output.

The parser writes to disk a file named `demoId.json`.

In [1]:
from csgo.parser import DemoParser

# Create parser object
# Set log=True above if you want to produce a logfile for the parser
demo_parser = DemoParser(
    demofile = "og-vs-natus-vincere-m1-dust2.dem", 
    demo_id = "OG-NaVi-BLAST2020", 
    parse_rate=128, 
    trade_time=5, 
    buy_style="hltv"
)


# Parse the demofile, output results to dictionary
data = demo_parser.parse()

14:48:19 [INFO] Go version>=1.14.0
14:48:19 [INFO] Initialized CSGODemoParser with demofile /home/peter/Downloads/csgo-tests/og-vs-natus-vincere-m1-dust2.dem
14:48:19 [INFO] Setting demo id to OG-NaVi-BLAST2020
14:48:19 [INFO] Setting parse rate to 128
14:48:19 [INFO] Setting trade time to 5
14:48:19 [INFO] Setting buy style to hltv
14:48:19 [INFO] Rollup damages set to False
14:48:19 [INFO] Parse frames set to True
14:48:19 [INFO] Running Golang parser from /home/peter/.pyenv/versions/3.8.4/lib/python3.8/site-packages/csgo-0.1-py3.8.egg/csgo/parser/
14:48:19 [INFO] Looking for file at /home/peter/Downloads/csgo-tests/og-vs-natus-vincere-m1-dust2.dem
14:48:28 [INFO] Wrote demo parse output to OG-NaVi-BLAST2020.json
14:48:28 [INFO] Reading in JSON from OG-NaVi-BLAST2020.json
14:48:29 [INFO] JSON data loaded, available in the `json` attribute to parser
14:48:29 [INFO] Successfully parsed JSON output
14:48:29 [INFO] Successfully returned JSON output


# Accessing the parsed data
Next, we can access our parsed data. The default data output is the JSON-like format of a Python dictionary. We describe the structure in the [documentation](https://github.com/pnxenopoulos/csgo/tree/main/csgo/docs). We show the results in the following cells.

In [3]:
for k in data.keys():
    if k != "gameRounds":
        print(k + ": " + str(data[k]))

matchID: OG-NaVi-BLAST2020
clientName: GOTV Demo
mapName: de_dust2
tickRate: 127
playbackTicks: 466670
parserParameters: {'parseRate': 128, 'parseFrames': False, 'tradeTime': 5, 'roundBuyStyle': 'hltv', 'damagesRolledUp': False}
serverVars: {'cashBombDefused': 0, 'cashBombPlanted': 0, 'cashTeamTWinBomb': 0, 'cashWinDefuse': 3500, 'cashWinTimeRunOut': 0, 'cashWinElimination': 0, 'cashPlayerKilledDefault': 0, 'cashTeamLoserBonus': 0, 'cashTeamLoserBonusConsecutive': 0, 'roundTime': 0, 'roundTimeDefuse': 2, 'roundRestartDelay': 5, 'freezeTime': 20, 'buyTime': 20, 'bombTimer': 0, 'maxRounds': 30, 'timeoutsAllowed': 4, 'coachingAllowed': 1}
matchPhases: {'announcementLastRoundHalf': [239899], 'announcementFinalRound': None, 'announcementMatchStarted': None, 'roundStarted': [890, 7167, 9177, 9308, 43817, 60608, 73511, 84662, 100420, 118340, 135431, 153351, 169400, 181861, 192298, 209674, 227594, 239899, 259186, 294862, 318686, 339215, 346620, 363966, 389948, 410601, 429300, 441487], 'roundEn

## Event Data
The parser contains data on kills, damages, grenades, bomb events, weapon fires and flashes, which we show below, in order.

In [15]:
for k in data["gameRounds"][7]["kills"][0].keys():
        print(k + ": " + str(data["gameRounds"][7]["kills"][0][k]))

tick: 88285
second: 8.37007874015748
attackerSteamID: 76561198037812456
attackerName: ISSAA
attackerTeam: OGEsports
attackerSide: CT
attackerX: -415.947265625
attackerY: 1627.2728271484375
attackerZ: -126.26683044433594
attackerAreaID: 6699
attackerAreaName: MidDoors
attackerViewX: 275.5865478515625
attackerViewY: 354.08935546875
victimSteamID: 76561198034202275
victimName: s1mple
victimTeam: Natus Vincere
victimSide: T
victimX: -268.5945129394531
victimY: 324.79052734375
victimZ: -0.4272732734680176
victimAreaID: 1714
victimAreaName: TopofMid
victimViewX: 97.09716796875
victimViewY: 4.7406005859375
assisterSteamID: None
assisterName: None
assisterTeam: None
assisterSide: None
isSuicide: False
isTeamkill: False
isWallbang: False
penetratedObjects: 0
isFirstKill: True
isHeadshot: True
victimBlinded: True
attackerBlinded: False
flashThrowerSteamID: 76561198016432560
flashThrowerName: mantuu
flashThrowerTeam: OGEsports
flashThrowerSide: CT
noScope: False
thruSmoke: False
distance: 1316.81

In [14]:
for k in data["gameRounds"][7]["damages"][0].keys():
        print(k + ": " + str(data["gameRounds"][7]["damages"][0][k]))

tick: 88216
second: 7.826771653543307
attackerSteamID: 76561198037812456
attackerName: ISSAA
attackerTeam: OGEsports
attackerSide: CT
attackerX: -411.76348876953125
attackerY: 1626.341064453125
attackerZ: -126.3670425415039
attackerAreaID: 6699
attackerAreaName: MidDoors
attackerViewX: 274.3011474609375
attackerViewY: 355.4461669921875
attackerStrafe: False
victimSteamID: 76561198034202275
victimName: s1mple
victimTeam: Natus Vincere
victimSide: T
victimX: -308.9562683105469
victimY: 320.0276794433594
victimZ: 0.9765335321426392
victimAreaID: 1714
victimAreaName: TopofMid
victimViewX: 94.647216796875
victimViewY: 2.5653076171875
weapon: M4A4
hpDamage: 21
hpDamageTaken: 21
armorDamage: 4
armorDamageTaken: 4
hitGroup: Chest


In [8]:
for k in data["gameRounds"][7]["grenades"][0].keys():
        print(k + ": " + str(data["gameRounds"][7]["grenades"][0][k]))

throwTick: 87276
destroyTick: 89832
throwSecond: 0.4251968503937008
destroySecond: 20.551181102362204
throwerSteamID: 76561198016432560
throwerName: mantuu
throwerTeam: OGEsports
throwerSide: CT
throwerX: 283.96875
throwerY: 2309.71875
throwerZ: -54.125
throwerAreaID: 8781
throwerAreaName: ExtendedA
grenadeType: Smoke Grenade
grenadeX: -367.34375
grenadeY: 2220.75
grenadeZ: -122.78125
grenadeAreaID: 8492
grenadeAreaName: MidDoors
UniqueID: 5709541018789381327


In [10]:
for k in data["gameRounds"][6]["bombEvents"][0].keys():
        print(k + ": " + str(data["gameRounds"][6]["bombEvents"][0][k]))

tick: 81271
second: 40.94488188976378
playerSteamID: 76561198116523276
playerName: flamie
playerTeam: Natus Vincere
playerX: -1551.3118896484375
playerY: 2514.281005859375
playerZ: 1.9367716312408447
bombAction: plant_begin
bombSite: B


In [13]:
for k in data["gameRounds"][7]["weaponFires"][0].keys():
        print(k + ": " + str(data["gameRounds"][7]["weaponFires"][30][k]))

tick: 91221
second: 31.488188976377952
playerSteamID: 76561198121220486
playerName: Perfecto
playerTeam: Natus Vincere
playerSide: T
playerX: -1935.073974609375
playerY: 1204.62890625
playerZ: 32.03125
playerAreaID: 8223
playerAreaName: UpperTunnel
playerViewX: 91.25244140625
playerViewY: 5.7293701171875
playerStrafe: False
weapon: FAMAS


In [17]:
for k in data["gameRounds"][7]["flashes"][0].keys():
        print(k + ": " + str(data["gameRounds"][7]["flashes"][0][k]))

tick: 88121
second: 7.078740157480315
attackerSteamID: 76561198116523276
attackerName: flamie
attackerTeam: Natus Vincere
attackerSide: T
attackerX: 555.159423828125
attackerY: 162.18075561523438
attackerZ: -1.203836441040039
attackerAreaID: 8924
attackerAreaName: OutsideLong
attackerViewX: 73.465576171875
attackerViewY: 2.3565673828125
playerSteamID: 76561198114929868
playerName: valde
playerTeam: OGEsports
playerSide: CT
playerX: 1654.9215087890625
playerY: 1925.246826171875
playerZ: 0.5459556579589844
playerAreaID: 4385
playerAreaName: LongA
playerViewX: 215.2166748046875
playerViewY: 331.534423828125
flashDuration: 0.450503968


## Event Data as DataFrames

We can also parse the data into dataframes.

In [18]:
df = demo_parser.parse(return_type="df")
df.keys()

14:53:10 [INFO] Running Golang parser from /home/peter/.pyenv/versions/3.8.4/lib/python3.8/site-packages/csgo-0.1-py3.8.egg/csgo/parser/
14:53:10 [INFO] Looking for file at /home/peter/Downloads/csgo-tests/og-vs-natus-vincere-m1-dust2.dem
14:53:23 [INFO] Wrote demo parse output to OG-NaVi-BLAST2020.json
14:53:23 [INFO] Reading in JSON from OG-NaVi-BLAST2020.json
14:53:24 [INFO] JSON data loaded, available in the `json` attribute to parser
14:53:24 [INFO] Successfully parsed JSON output
14:53:24 [INFO] Successfully returned JSON output
14:53:24 [INFO] Parsed rounds to Pandas DataFrame
14:53:24 [INFO] Parsed kills to Pandas DataFrame
14:53:24 [INFO] Parsed damages to Pandas DataFrame
14:53:24 [INFO] Parsed grenades to Pandas DataFrame
14:53:24 [INFO] Parsed flashes to Pandas DataFrame
14:53:24 [INFO] Parsed weapon fires to Pandas DataFrame
14:53:24 [INFO] Parsed bomb_events to Pandas DataFrame
14:53:24 [INFO] Parsed frames to Pandas DataFrame
14:53:24 [INFO] Parsed player frames to Panda

dict_keys(['matchID', 'clientName', 'mapName', 'tickRate', 'playbackTicks', 'rounds', 'kills', 'damages', 'grenades', 'flashes', 'weaponFires', 'bombEvents', 'frames', 'playerFrames'])

In [19]:
df["kills"]

,tick,second,attackerSteamID,attackerName,attackerTeam,attackerSide,attackerX,attackerY,attackerZ,attackerAreaID,...,thruSmoke,distance,isTrade,playerTradedName,playerTradedTeam,playerTradedSteamID,weapon,roundNum,matchID,mapName
0,3864,3.267717,7.656120e+16,NBK-,OGEsports,T,-898.865112,-767.774658,119.389297,6993.0,...,False,32.032988,False,None,None,NaN,Knife,1,OG-NaVi-BLAST2020,de_dust2
1,5402,15.377953,7.656120e+16,mantuu,OGEsports,T,1372.470337,1408.143555,-8.232350,4184.0,...,False,49.255150,False,None,None,NaN,Knife,1,OG-NaVi-BLAST2020,de_dust2
2,6007,20.141732,7.656120e+16,Boombl4,Natus Vincere,CT,1557.202393,2209.131104,-8.981688,4179.0,...,False,48.994065,False,None,None,NaN,Knife,1,OG-NaVi-BLAST2020,de_dust2
3,6116,21.000000,7.656120e+16,mantuu,OGEsports,T,878.195435,2149.758301,-23.769341,8677.0,...,False,32.290789,False,None,None,NaN,Knife,1,OG-NaVi-BLAST2020,de_dust2
4,6265,22.173228,7.656120e+16,NBK-,OGEsports,T,1482.418701,2158.469482,-9.959195,4179.0,...,False,33.627793,False,None,None,NaN,Knife,1,OG-NaVi-BLAST2020,de_dust2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,456139,4.181102,7.656120e+16,electronic,Natus Vincere,CT,-484.418335,1847.642334,-123.553299,5220.0,...,False,221.081781,False,None,None,NaN,M4A4,28,OG-NaVi-BLAST2020,de_dust2
167,458062,19.322835,7.656120e+16,valde,OGEsports,T,-2025.999146,1511.807251,33.910797,8063.0,...,False,602.142870,False,None,None,NaN,AK-47,28,OG-NaVi-BLAST2020,de_dust2
168,458223,20.590551,7.656120e+16,valde,OGEsports,T,-2015.828979,1507.713623,33.111588,8063.0,...,False,251.518662,False,None,None,NaN,AK-47,28,OG-NaVi-BLAST2020,de_dust2
169,458294,21.149606,7.656120e+16,electronic,Natus Vincere,CT,-1753.573730,1118.690674,31.989418,1245.0,...,False,467.218752,True,Boombl4,Natus Vincere,7.656120e+16,M4A4,28,OG-NaVi-BLAST2020,de_dust2


In [20]:
df["damages"]

,tick,second,attackerSteamID,attackerName,attackerTeam,attackerSide,attackerX,attackerY,attackerZ,attackerAreaID,...,victimViewY,weapon,hpDamage,hpDamageTaken,armorDamage,armorDamageTaken,hitGroup,roundNum,matchID,mapName
0,3570,0.952756,7.656120e+16,NBK-,OGEsports,T,-1176.274048,-850.651001,115.850540,8826.0,...,11.024780,Knife,30,30,2,2,Generic,1,OG-NaVi-BLAST2020,de_dust2
1,3634,1.456693,7.656120e+16,NBK-,OGEsports,T,-1197.305298,-820.531494,115.466309,8826.0,...,11.024780,Knife,30,30,2,2,Generic,1,OG-NaVi-BLAST2020,de_dust2
2,3800,2.763780,7.656120e+16,NBK-,OGEsports,T,-962.809326,-777.490967,118.282295,6993.0,...,11.024780,Knife,30,30,2,2,Generic,1,OG-NaVi-BLAST2020,de_dust2
3,3864,3.267717,7.656120e+16,NBK-,OGEsports,T,-898.865112,-767.774658,119.389297,6993.0,...,11.024780,Knife,30,10,2,2,Generic,1,OG-NaVi-BLAST2020,de_dust2
4,5295,14.535433,7.656120e+16,flamie,Natus Vincere,CT,1390.043091,1362.169434,-10.142410,4184.0,...,9.569092,Knife,55,55,4,4,Generic,1,OG-NaVi-BLAST2020,de_dust2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,458191,20.338583,7.656120e+16,Boombl4,Natus Vincere,CT,-1873.927979,1302.483154,33.934990,1230.0,...,0.351562,AK-47,27,27,4,4,Chest,28,OG-NaVi-BLAST2020,de_dust2
734,458217,20.543307,7.656120e+16,Boombl4,Natus Vincere,CT,-1883.084595,1294.227783,32.957306,1230.0,...,0.527344,AK-47,27,27,4,4,Chest,28,OG-NaVi-BLAST2020,de_dust2
735,458223,20.590551,7.656120e+16,valde,OGEsports,T,-2015.828979,1507.713623,33.111588,8063.0,...,2.614746,AK-47,110,100,16,16,Head,28,OG-NaVi-BLAST2020,de_dust2
736,458294,21.149606,7.656120e+16,electronic,Natus Vincere,CT,-1753.573730,1118.690674,31.989418,1245.0,...,0.834961,M4A4,89,46,19,19,Head,28,OG-NaVi-BLAST2020,de_dust2
